In [23]:
import pandas as pd

# Specify the path to your CSV file
file_path = 'new_file.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)



In [24]:
! pip install -U accelerate
! pip install -U transformers

In [25]:
import accelerate
print(accelerate.__version__)

1.2.0


In [26]:
df

,Tier 1,Tier 2,URL,Title,Description,Keywords,Site Name,Content,License,Language,Tier 3,Tier 4,combined_text
0,Automotive,driving,https://www.techradar.com/best/best-dash-cam,The best dash cam 2024: The latest car cameras...,"Whatever your needs and budget, we've round up...",driving,TechRadar,best dash cam nextbase garmin thinkware purcha...,cc by sa,en,NaN,NaN,The best dash cam 2024: The latest car cameras...
1,Automotive,driving,https://www.amazon.com/Camera-Built-170%C2%B0W...,"Amazon.com: Dash Camera for Cars,4K Full UHD C...","Amazon.com: Dash Camera for Cars,4K Full UHD C...",driving,Unknown Site,offer easy convenient return least one free re...,cc by sa,en,NaN,NaN,"Amazon.com: Dash Camera for Cars,4K Full UHD C..."
2,Automotive,driving,https://matrackinc.com/dash-cams-with-cloud-st...,Top 6 Dash cams with Local and Cloud Storage -...,est dash cams with local and cloud storage for...,driving,Matrackinc -,wed love send info fill form query one team me...,cc by sa,en,NaN,NaN,Top 6 Dash cams with Local and Cloud Storage -...
3,Automotive,hybrid,https://www.hikvision.com/en/products/onboard-...,Industry Dash Camera - Dash Cameras - Hikvision,Hikvision dashcams provide a lightweight solut...,hybrid,Hikvision,login global en product network product networ...,cc by sa,en,NaN,NaN,Industry Dash Camera - Dash Cameras - Hikvisio...
4,Automotive,hybrid,https://www.hikvision.com/en/,Hikvision - Leading Video AIoT Solutions & Ser...,Hikvision provides top-of-the-line AIoT soluti...,hybrid,Hikvision,login global en product network product networ...,cc by sa,en,NaN,NaN,Hikvision - Leading Video AIoT Solutions & Ser...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24507,Automotive,adventure,https://www.riders-share.com/type/scooter,Scooter Rentals & Moped Rentals - Riders Share,Rent a scooter or moped across the United Stat...,adventure,Unknown Site,step world motor scooter moped rental convenie...,cc,en,NaN,NaN,Scooter Rentals & Moped Rentals - Riders Share...
24508,Automotive,research,https://tvsabl.com/,"Best Two Wheeler in Bangladesh - Bike, Scooter...",: TVS Auto Bangladesh Limited - Find the best ...,research,TVS Auto Bangladesh Limited,imposing gaze thatll stop track sound thats ro...,cc,en,NaN,NaN,"Best Two Wheeler in Bangladesh - Bike, Scooter..."
24509,Automotive,styling,https://www.bikewale.com/new-bike-search/best-...,"Best scooty under 60,000 | Top scooters under ...","Looking to buy a scooty under 60,000? Here is ...",styling,BikeWale,rating parameter design styling reliability co...,cc,en,NaN,NaN,"Best scooty under 60,000 | Top scooters under ..."
24510,Automotive,employment,https://www.topgearpowersports.com/Inventory/A...,Scooters For Sale: Inventory at Top Gear Power...,Shop all in-stock Scooters inventory for sale ...,employment,Unknown Site,shop inventory sale dealership roselle illinoi...,cc,en,NaN,NaN,Scooters For Sale: Inventory at Top Gear Power...


In [27]:
import torch
print(torch.cuda.is_available())

True


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
X = df['combined_text']
y = df['Tier 1']

In [9]:
# Check if lengths match
print(len(X), len(y))  # Should output the same number

# Proceed with your vectorizer and model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data to feature vectors
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

24512 24512


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example dataset (replace with your DataFrame)
df = df.dropna(subset=['combined_text'])  # Remove rows with NaN in 'combined_text'

# Prepare data
X = df['combined_text'].tolist()
y = pd.factorize(df['Tier 2'])[0]  # Convert Tier 1 labels to numerical format

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a custom dataset for Hugging Face
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length=64)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_length=64)

# Load the model and move it to the correct device
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(y)))
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False,  # Disable pushing to Hugging Face hub
    report_to="none",   # Avoid extra logging if unnecessary
    fp16=torch.cuda.is_available()  # Use mixed precision training if on GPU
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

# Save the model
model.save_pretrained('./tier1_classifier')
tokenizer.save_pretrained('./tier1_classifier')

print("Model and tokenizer saved to './tier1_classifier'")

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Kaneki\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_30768\3984759120.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 10/7356 [00:02<29:30,  4.15it/s] 

{'loss': 5.2948, 'grad_norm': 11.078274726867676, 'learning_rate': 1.9975530179445354e-05, 'epoch': 0.0}


  0%|          | 21/7356 [00:05<22:37,  5.40it/s]

{'loss': 5.132, 'grad_norm': 8.310257911682129, 'learning_rate': 1.9951060358890702e-05, 'epoch': 0.01}


...

{'loss': 4.9826, 'grad_norm': 9.141681671142578, 'learning_rate': 1.9872213159325723e-05, 'epoch': 0.02}


  1%|          | 61/7356 [00:13<22:53,  5.31it/s]  

{'loss': 4.8716, 'grad_norm': 9.060290336608887, 'learning_rate': 1.9845024469820556e-05, 'epoch': 0.02}


  1%|          | 68/7356 [00:15<23:31,  5.16it/s]

: 

In [11]:
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

100%|██████████| 613/613 [02:36<00:00,  3.91it/s]

Evaluation Results: {'eval_loss': 1.0939775705337524, 'eval_runtime': 157.2537, 'eval_samples_per_second': 31.179, 'eval_steps_per_second': 3.898, 'epoch': 3.0}


In [12]:
from sklearn.metrics import accuracy_score

# Get predictions on the test dataset
predictions = trainer.predict(test_dataset)

# Extract predicted labels
predicted_labels = predictions.predictions.argmax(axis=1)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, predicted_labels)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

100%|██████████| 613/613 [02:13<00:00,  4.61it/s]

Test Accuracy: 75.57%


In [17]:
predictions.label_ids[1:10]

array([ 1, 28, 10, 13,  7, 18,  1, 21,  7], dtype=int64)

In [ ]:
predictions

In [13]:
test_dataset

In [22]:
# Create a mapping from numerical labels to their original string labels
label_mapping = {index: label for index, label in enumerate(set(df['Tier 1']))}

# Extract predicted labels
predicted_labels = predictions.predictions.argmax(axis=1)

# Create a DataFrame for better visualization
test_results = pd.DataFrame({
    'Content': X_test,                              # Original text content
    'True Label': [label_mapping[i] for i in y_test],  # Convert numerical labels to original text labels
    'Predicted Label': [label_mapping[i] for i in predicted_labels]  # Convert predicted numerical labels to original text labels
})

# Display the DataFrame
print(test_results.head())

                                             Content            True Label  \
0  Lincoln® Hybrid Electric Vehicles | Performanc...             Education   
1  Initial Registration as a Manufacturer for Pha...                   Law   
2  What Is the BRICS Group and Why Is It Expandin...          Food & Drink   
3  American Adoptions - American Adoptions - Amer...      Personal Finance   
4  Makesy® - shop fragrance oils, candle supplies...  Business and Finance   

        Predicted Label  
0             Education  
1                   Law  
2                   Law  
3      Personal Finance  
4  Business and Finance  


In [19]:
# Create a mapping of numerical labels to original labels
label_mapping = {index: label for index, label in enumerate(pd.factorize(df['Tier 1'])[1])}

# Print the mapping
print("Label Mapping (Numerical to Tier 1):")
for num_label, tier1_label in label_mapping.items():
    print(f"{num_label}: {tier1_label}")

Label Mapping (Numerical to Tier 1):
0: Automotive
1: Business and Finance
2: Books and Literature
3: Careers
4: Communication
5: Crime
6: Disasters
7: Education
8: Entertainment
9: Events
10: Family and Relationships
11: Fine Art
12: Food & Drink
13: Hobbies & Interests
14: Genres
15: Healthy Living
16: Holidays
17: Home & Garden
18: Law
19: Medical Health
20: Maps & Navigation
21: Personal Finance
22: Personal Celebrations & Life Events
23: Pets
24: Politics
25: Pop Culture
26: Productivity
27: Real Estate
28: Religion & Spirituality
29: Science
30: Sensitive Topics
31: Shopping
32: Style & Fashion
33: Technology & Computing
34: Travel
35: Video Gaming
36: War and Conflicts
37: Attractions


In [21]:
from neo4j import GraphDatabase
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Connect to Neo4j
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def query(self, query, parameters=None):
        with self.driver.session() as session:
            return session.run(query, parameters)

# Initialize Neo4j connection
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", password="kenkaneki")

# Step 2: Get Tier 2 candidates from Neo4j
def get_tier2_candidates_from_neo4j(predicted_tier1):
    query = """
    MATCH (t1:Tier1 {name: $tier1_name})-[:HAS]->(t2:Tier2)
    RETURN t2.name AS tier2_name
    """
    result = conn.query(query, parameters={"tier1_name": predicted_tier1})
    return [record["tier2_name"] for record in result]

# Step 3: Match combined_text with Tier 2 candidates
def match_tier2_with_similarity(combined_text, tier2_candidates):
    if not tier2_candidates:
        return "No Tier 2 Candidates Found"
    
    # Vectorize texts
    vectorizer = TfidfVectorizer()
    texts = [combined_text] + tier2_candidates
    tfidf_matrix = vectorizer.fit_transform(texts)
    
    # Compute cosine similarity
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    most_similar_idx = similarities.argmax()
    
    return tier2_candidates[most_similar_idx]

# Step 4: Predict Tier 2 for test data
tier2_predictions = []

for i, row in X_test.iterrows():  # Assuming test_df contains test data
    predicted_tier1 = row['Predicted Tier 1']  # From Tier 1 classification
    combined_text = row['combined_text']
    
    # Fetch Tier 2 candidates from Neo4j
    tier2_candidates = get_tier2_candidates_from_neo4j(predicted_tier1)
    
    # Match and predict Tier 2
    predicted_tier2 = match_tier2_with_similarity(combined_text, tier2_candidates)
    tier2_predictions.append(predicted_tier2)

# Add Tier 2 predictions to DataFrame
X_test['Predicted Tier 2'] = tier2_predictions

# Display results
print(X_test[['combined_text', 'Predicted Tier 1', 'Predicted Tier 2']])

# Close Neo4j connection
conn.close()

AttributeError: 'list' object has no attribute 'iterrows'